<div class="alert alert-block alert-warning">
    <p><b>READ THIS:</b></p>
    <p>Before using the Notebook below, take a look at the recently proivded Esri Notebook sample, <a href="https://developers.arcgis.com/python/samples/clone-storymap-version2/">Copying an ArcGIS StoryMap item to another organization</a>. This example takes advantages of new features (and avoids bugs) in the ArcGIS API for Python, and represents a better way to clone ArcGIS StoryMaps than was previously available; however, it takes a different approach on what referenced content to include in the copy.</p>
    <p>Also, please condsider visting the Esri Community ArcGIS Online Idea place and kudoing: <a href="https://community.esri.com/t5/arcgis-online-ideas/transfer-or-clone-arcgis-storymaps-with-greater/idi-p/1257287">Transfer or Clone ArcGIS StoryMaps with greater ease</a>.</p>
</div>

<div class="alert alert-block alert-warning">
    <p><b>Warning: </b>Due to bugs present in the ArcGIS API for Python versions 1.9.x to 2.0.x, it is recommended to run this Notebook with version 1.8.4 of the API. The bug mainfests as the clone_items() method getting stuck and genereating multiple clones, over and over, so that it never finsihes, and multiple copies accumulate on the target system.</p>
    <p>If you are running this as an ArcGIS Online Notebook, then choose runtime version 5.0, which uses version 1.8.4 of the API.</p>
</div>

# Clone an ArcGIS StoryMap

(Version 2022.02.13)

A Notebook to guide you through you through the steps of cloning an ArcGIS StoryMap. (It is not for cloning Classic StoryMap apps.)

The goal of this Notebook is to support situations where a "full" clone of StoryMap is required. Where the person creating the StoryMap wants to keep a copy for themselves, under their full control, however, they also need to provide a copy of the StoryMap to someone else, who will then have full control over that copy.

This Notebook defines "full" clone as cloning not only the StoryMap, but also any referenced content in the StoryMap that is owned by the same user as the StoryMap. Cloning referenced content is supported for the major content types (images, express maps, web maps, web scenes, feature layers, tile layers, etc.) For unsupported content types, the reference to the orignal item wil be carried through to the cloned StoryMap. Similarly, for referenced content that is not owned by the same user, the reference to the original item will be maintained in the clone.

The sharing settings for the StoryMap and its related content are not currently cloned. Cloned content is private to the owner.

## Some Additional Notes
* The code in this Notebook is designed for readability and understanding of the various steps. It is not optimized.
* It is generally recommended that the user running this Notebook have the built-in Administrator role. The user running the Notebook needs to have full access to the StoryMap (and any referenced content) to create a full copy. If the user only has view access to the StoryMap, then the Notebook can only clone the published version of the StoryMap. Any unpublished edits in the draft version will not be accessible, and, therefore, not included in the cloned StoryMap. (The latter situation can occur even when running as an Administrator, if you are not an Administrator on the system from which the StoryMap is beling cloned; for example, when you are cloning a StoryMap from a public ArcGIS Online account.)
* If a StoryMap is shared with a Shared Update group, then the user running this Notebook need only be a member of that group with access to edit the StoryMap for the Notebook to be able to clone any draft version of the StoryMap, in addition to the published version.
* Cloning a StoryMap produces a new item, which means a new URL. For certain use cases, you may want to change the ownershp of the original StoryMap (and appropriate referenced content) to the person for whom the clone is being made, so that references to the original URL are maintained, and then clone the StoryMap back to the original owner. For example, if a student makes StoryMap on behalf of faculty member, and the faculty member has already been distributing or linking to that StoryMap's URL, then it would be best to make the faculty member the owner of the original StoryMap, and produce a clone of it for the student.
* The sharing settings for the original StoryMap are not carried over to the clone. The cloned StoryMap is initially only visible to its owner.
* The code below assumes you are using built-in ArcGIS accounts to connect to your source and target GIS systems. If either or both use enterprise logins instead, like SAML or ADFS, then you will need to modify the connection process for your particular systems' configurations. For example, with SAML, you are usually redirected to a web page to login, so you do not put your username and password in the Notebook, as you do for a built-in account. For an overview of using different authentication schemes with the ArcGIS API for Python, please see [Working with different authentication schemes](https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/). 
* Cloning of StoryMaps with Custom Themes is not currently fully supported. The cloned StoryMap will end up using a default StoryMap Theme. You can separately clone or recreate your Custom Theme and apply it to the cloned StoryMap.

## Warnings and Errors
* The code may generate warning messages when it isn't sure what to do, such as when it encounters a version of StoryMaps that it wasn't previously tested against.
* If you have a very complex or large StoryMap, then the clone_items() method may not be able to complete within the ArcGIS Online execution time limit. Sometimes buried in the error messages ArcGIS returns there is mention of a timeout, but not always. If you get an error in one of the cells that uses clone_items(), and it has been running for more than 10 minutes, then it is probably an error related to a timeout. Unfortunately, it is not possible to override this limit. (If you're comfortable with Python, and clone_items() is failing when working on multiple items, then you could try breaking up the call into multiple calls that clone fewer items at one time.)

# Initialization

In [ ]:
import arcgis
from arcgis import GIS
from arcgis import mapping
from arcgis import features
import datetime
import json
from pkg_resources import parse_version

# Minimum and maxium known versions of ArcGIS StoryMaps for which this script is known to work.
# (Note that older versions of StoryMaps can be updated to the latest verison by re-publishing them.)
minimum_storymap_version = parse_version('20.0.0')
maximum_storymap_version = parse_version('23.2.385')    # lastest known version, circa January 2023.

In [ ]:
# Check that the version of the ArcGIS API for Python in the current environment. 
if( parse_version(arcgis.__version__) < parse_version('1.7.0') ):
    print("This Notebook is known to work in version 1.7.0-1.8.4 of the ArcGIS API for Python.")
    print("You are using an earlier version of the API {0}. Proceed at your own risk...".format(arcgis.__version__))
    print("Location:", arcgis.__file__)
elif( parse_version(arcgis.__version__) > parse_version('1.8.4') and parse_version(arcgis.__version__) < parse_version('2.0.0')):
    print("This Notebook is known to work in version 1.7.0-1.8.4 of the ArcGIS API for Python.")
    print("You are using a version {0}, which is known to include bugs that prevent this Notebook from completing its task.".format(arcgis.__version__))
    print("Location:", arcgis.__file__)
else:
    print("Environment is using ArcGIS API for Python version:", arcgis.__version__)

# Provide Information for Cloning

<div class="alert alert-block alert-info">
    <b>Note: </b>This is the only section in which you should need to edit the code.
</div>

## StoryMap to be Cloned

In [ ]:
# Enter the Item ID for the StoryMap to be cloned (e.g., '1a8199d7b768471d9af0e7be75d9009e')
source_storymap_itemId = 'item_id'

## Source GIS Connection Info

Typically this is the ArcGIS Online organization that hosts the StoryMap which is being cloned, and for which the user running this Notebook is an Administrator.

If the StoryMap to be cloned is hosted on an organization of which the user running the Notebook is not a member, then as long as the StoryMap and its content are shared publicly, then this Notebook will clone the published version of the StoryMap, but not any unpublished edits in the draft version of the StoryMap.

In [ ]:
# Define the source GIS which will be used to access the StoryMap to be cloned.
# Fill in the login information for the source GIS account here, or set use_builtin to True to authenticate via ArcGIS Pro or ArcGIS Online:
source_portal_url = ''
source_username = ''
source_password = ''
source_use_builtin = False

## Target GIS Connection Info

This is the ArcGIS Online organization to which the StoryMap is being cloned.

<div class="alert alert-block alert-info">
    <b>Note: </b>If you are creating a new account on the target organization to use for this cloning process, make sure you log in to that account once directly in your browser. Simply creating it as an administrator is not enough. You need to login to it once to complete its configuration, before you can use it in a Notebook.
</div>

In [ ]:
# Define the target GIS (where the story map should be copied to)
# Fill in the login information for the source GIS account here, or set use_builtin to True to authenticate via ArcGIS Pro or ArcGIS Online:
target_portal_url = ''   # URL for your ArcGIS for Student Use ArcGIS Online instance.
tagert_username = ''
target_password = ''
target_use_builtin = False

# Clone an ArcGIS StoryMap

Begin the process of cloning the specified ArcGIS StoryMap.

<div class="alert alert-block alert-info">
    <b>Note: </b>Normally you do not need to edit any code below this point.
</div>

## Setup GIS Connections

In [ ]:
# A function for connecting to an ArcGIS Online instance.
def gis_login(portal_url='', username='', password='', use_builtin=False):
    try:
        if use_builtin:
            gis = GIS('home')
        else:
            gis = GIS(url = portal_url,username=username, password=password) 
        print( 'Login successful.' )
        print( '    url:      ' + gis.url)
        # Name is only available for organizational subscriptions, not public ArcGIS Oline accounts.
        if( 'name' in gis.properties.user):
            print( '    server:   ' + gis.properties.name )
        print( '    user:     ' + gis.properties.user.username )
        # Role only available for organizational subscriptions, not public ArcGIS Oline accounts.
        if( 'role' in gis.properties.user):
            print( '    role:     ' + gis.properties.user.role )
        print( '    provider: ' + gis.properties.user.provider )
    except Exception as e:
        print('Login error.')
        print(e)
        gis = None
    return(gis)

In [ ]:
# Connect to source GIS instance.
source_gis = gis_login(portal_url=source_portal_url, username=source_username, password=source_password, use_builtin=source_use_builtin)

In [ ]:
# Connect to target GIS instance.
target_gis = gis_login(portal_url=target_portal_url, username=tagert_username, password=target_password, use_builtin=target_use_builtin)

## Create Tag for Cloned Items

The tag can help with searching to find all the content created as a result of the cloning. Note that clone_items() will also add tags containing item IDs for some items it produces, in order to help associate them with their originals.

In [ ]:
# Create a Tag to add to cloned items to help keep track of things, in case you need to undo something.
# The tag incorporates the current time at which this Notebook is being run.
# (While the system permits you to put a ":" in tag, do not do so! It is the delimiter for keywords used
# in querying in ArcGIS onilne (e.g., query="tags:Demo"), so using a ":" inside a tag will make it 
# difficult to search for.)
process_timestamp_tag = 'StoryMap_Clone_Created_' + datetime.datetime.now(datetime.timezone.utc).strftime("%Y-%m-%d_%H.%M.%S.%f_UTC")
print('process_timestamp_tag:', process_timestamp_tag)

In [ ]:
# Check to make sure there are no items already on the target GIS using this tag.
items = target_gis.content.search("tags:"+process_timestamp_tag)
if( len(items) > 0 ):
    # Content with this tag really shouldn't exist already, so if it does, then you probably should start over and run the cell above again to get a new tag.
    print("ERROR: Content with tag already exists on target GIS!")
    print(items)
else:
    print("No content with tag exists on target GIS.")

## Retrive the StoryMap to Clone

In [ ]:
# Get StoryMap to clone. 
storymap_item = source_gis.content.get(source_storymap_itemId)
storymap_item

In [ ]:
# Set source_user to owner of StoryMap; only referenced items also owned by the same user should also be cloned.
source_user = storymap_item['owner']
print('StoryMap owned by:', source_user)

In [ ]:
# Determine if user running script has access to unpublished edits for StoryMap, or only the published version, 
# in which case those edits will not be included in the clone.
#
# Unpublished edits are inaccessible to the user running the script when:
# * user is not a member of the ArcGIS Online organization hosting the StoryMap, or
# * user is from the same org, but is not an Admin, nor the owner of the StoryMap

# Get user item for StoryMap owner.
storymap_user = source_gis.users.get(source_user)

# Display warning to user, if this script will not be able to clone unpublished edits.
# Also set flag for access to those draft changes, to that thigns can be handled appropriately later on.
if( 
    ('orgId' not in storymap_user) or
    (source_gis.properties.user.role != 'org_admin' and source_gis.properties.user.username != source_user )
):
    print("WARNING: Your account does not have access to unpublished edits in the StoryMap. If any are present, then they will not be included in the clone.")
    print("    Your account:", source_gis.properties.user.username)
    draft_accessible = False
else:
    print("Your account has access to unpublished edits in the StoryMap. If any are present, then will be included in the clone.")
    draft_accessible = True

In [ ]:
# For reference, dispaly the StoryMap's typeKeywords
print("StoryMap's typeKeywords", storymap_item['typeKeywords'])

In [ ]:
# Check StoryMap's typeKeywords to ensure it fits the model this Notebook was built upon.

# Check that it is an arcgis-storymaps item.
if( 'arcgis-storymaps' in storymap_item['typeKeywords'] ):
    print("arcgis-storymaps typeKeyword is present.")
else:
    print("WARNING: 'arcgis-storymaps' not present in typeKeywords!")

# Check that status is one of the known typeKeywords.
status = [ x for x in storymap_item['typeKeywords'] if x.startswith(('smstatusdraft','smstatuspublished','smstatusunpublishedchanges'), 0)]
if( status ):
    print(status[0], 'is a known StoryMap status.')
else:
    print("WARNING: no known StoryMap status typeKeyword is present!")
    
# Check that StoryMap's versions are present, and are one of the known versions.
smdraftversion = [ x for x in storymap_item['typeKeywords'] if x.startswith('smversiondraft', 0)]
if( smdraftversion ):
    version = parse_version(smdraftversion[0][15:])
    if( version >= minimum_storymap_version and version <= maximum_storymap_version ):
        print('smdraftversion:', version, 'is a tested StoryMap version.')
    elif( version < minimum_storymap_version ):
        print("WARNING: StoryMap has an untested smdraftversion:", version)
        print("WARNING: It is recommended that you re-publish the StoryMap so that it is updated to the current version of ArcGIS StoryMaps.")
    else:
        print("WARNING: StoryMap has an untested smdraftversion:", version)
        print("WARNING: This version is newer then the most recent release with which this script was tested, and may work perfectly fine.")
else:
    print("WARNING: no 'smversiondraft' typeKeyword found.")
# A published version will only be presnet if StoryMap has been published.
if( any(status in ['smstatuspublished','smstatusunpublishedchanges'] for status in storymap_item['typeKeywords']) ):
    smversionpublished = [ x for x in storymap_item['typeKeywords'] if x.startswith('smversionpublished', 0)]
    if( smversionpublished ):
        version = parse_version(smversionpublished[0][19:])
        if( version >= minimum_storymap_version and version <= maximum_storymap_version ):
            print('smversionpublished:', version, 'is a tested StoryMap version.')
        elif( version < minimum_storymap_version ):
            print("WARNING: StoryMap has an untested smversionpublished:", version)
            print("WARNING: It is recommended that you re-publish the StoryMap so that it is updated to the current version of ArcGIS StoryMaps.")
        else: 
            print("WARNING: StoryMap has an untested smversionpublished:", version)
            print("WARNING: This version is newer then the most recent release with which this script was tested, and may work perfectly fine.")
    else:
        print("WARNING: no 'smversionpublished' typeKeyword found.")

## Identify Referenced Content in StoryMap

Generate a list of item IDs for the referenced content in both, if applicable, the published and draft versions of the Storymap.

In [ ]:
# Retrieve the StoryMap's draft json Resource ID (or file name) from the smdraftresourceid keyword in the item's typeKeywords.
smdraftresourceid = [x[18:] for x in storymap_item['typeKeywords'] if x.startswith('smdraftresourceid:',0)][0]
smdraftresourceid

In [ ]:
# Function for generating a list of itemIds for referenced resources for supported types (i.e., web maps, web scenes) in a StoryMap
def get_resource_itemIds(resources):
    wm_itemIds = []
    for key, val in resources.items():
        print()
        if( val['type'] == 'webmap' ):
            
            # Obtain referenced item.
            itemId = val['data']['itemId']
            items = source_gis.content.search('id:' + itemId, outside_org = True)
            
            # If item is accessible to user running Notebook, then process the item referenced.
            if( len(items) == 1 ):
                item = source_gis.content.get(itemId)
                # Only track itemIds for swizzling for webmap resources owned by the same user (i.e., source_user.)
                if(item['owner'] == source_user):
                    print( item['id'], val['type'] )
                    print( item['type'], item['typeKeywords'] )
                    # Is webmap item a Web Map?
                    if( 'Web Map' in item['typeKeywords'] ):
                        print( ">>>> Adding Web Map to wm_itemIDs map" )
                        wm_itemIds.append(item['id'])
                    # Is webmap item a Web Scene?
                    elif( 'Web Scene' in item['typeKeywords'] ):
                        print( ">>>> Adding Web Scene to wm_itemIDs map" )
                        wm_itemIds.append(item['id'])
                    # Else is an unsupported webmap type... what is it?
                    else:
                        print( ">>>> WARNING: Unsupported webmap type" )
                else:
                    print( item['id'], item['owner'])
                    print(">>>> Not owned by source_user")
                    
            elif( len(items) == 0 ):
                # User running script is not able to accessed the referenced item, so return an error, as clone will not be complete.
                print("ERROR: Referenced item is not accessible, so cannot include in clone. ID =", itemId)
                
            else:
                print("ERROR: unexpected number of results returned from search", len(items))
                
        elif( val['type'] == 'expressmap' or val['type'] == 'express-image-data' or val['type'] == 'image' or val['type'] == 'video' or val['type'] == 'story-theme' or val['type'] == 'audio'):
            # These "built-in" types are cloned along with the StoryMap, so they have no themeItemId to track for them.
            # TODO: Add support for custom StoryMap Themes, as they do need to be cloned themselves, and then applied to cloned StoryMap.
            # Only custom themes have a value for themeItemId.
            print(val['type'])
            print(">>>> Inclusive resource type, no itemId to track")
        else:
            # Any type we haven't yet dealt with...
            print(val['type'])
            print(">>>> WARNING: 'type' is not supported")
    return(wm_itemIds)

In [ ]:
# Get list of resource itemIds in draft JSON, if draft is accessible by user running Notebook.
print("Draft JSON itemIds:")
if( draft_accessible ):
    wm_itemIds = get_resource_itemIds(storymap_item.resources.get(smdraftresourceid)['resources'])
else:
    print("You do not have access to StoryMap's unpublished edits. They will not be cloned.")
    wm_itemIds = []                               

In [ ]:
# If StoryMap is published, then add to list the list of resource itemIds in published JSON.
if( any(status in ['smstatuspublished','smstatusunpublishedchanges'] for status in storymap_item['typeKeywords']) ):
    print("\nPublished JSON itemIds:")
    wm_itemIds.extend(get_resource_itemIds(storymap_item.get_data()['resources']))
else:
    print("StoryMap is not published.")

In [ ]:
# Show complete list of webmap itemIds. 
print("\nComplete list of webmap itemIds:")
print(wm_itemIds)

# Reduce list to unique set of webmap itemIds.
wm_itemIds = list(set(wm_itemIds))

# Show unique list of itemIds.
print("\nUnique list of webmap itemIds:")
print(wm_itemIds)

## Retrieve Referenced Layers

Get a list of feature and tile layers, which are referenced by Web Maps and Web Scenes referenced in the Story Map.

In [ ]:
# Function to extract feature layer itemIds owned by source user from layers in Web Maps and Web Scenes.
def extract_fl_itemIds( layers ):
    for layer in layers:
        print()
        print("layer:", layer['id'], layer['layerType'])
        # Is layer an "ArcGISFeatureLayer" or 'ArcGISTiledMapServiceLayer', and has an itemId?
        if( layer['layerType'] in ['ArcGISFeatureLayer','ArcGISTiledMapServiceLayer'] and 'itemId' in layer.keys() ):
            fl_item = source_gis.content.get(layer['itemId'])
            # Is feature layer owned by source_user
            if( fl_item['owner'] == source_user ):
                print("layer itemId:", layer['itemId'])
                fl_itemIds_to_clone.append(layer['itemId'])
                print(">>>> CLONE")
            else:
                print("layer itemId:", layer['itemId'])
                fl_itemIds_do_not_clone.append(layer['itemId'])
                print(">>>> PASS AS-IS: Not owned by source_user.")
        # Is layer a "GroupLayer"?
        elif( layer['layerType'] == 'GroupLayer' ):
            print("Group Layer:", layer['title'] )
            extract_fl_itemIds(layer['layers'])
        else:
            print(">>>> IGNORE: Not an ArcGISFeatureLayer with an itemId.")

In [ ]:
# Get list of Feature Layer itemIds in Web Maps for Feature Layers that are ArcGISFeatureLayer, have an itemId,
# and are owned by source_user. These are the Feature Layers which need to be cloned.
#
# Also get a list of Feature Layer itemIds in Web Maps for ones that do not need to be cloned, so that these can be 
# passed through as-is; the cloned StoryMap will point the same item in this case as the original StoryMap.
#
# Goal is to avoid cloning unnecessary duplicates of feature layers referenced by more than one map or xcene in Story, 
# and to avoid clonging feature layers not owned by the source_user (not their's to clone).

fl_itemIds_to_clone = []
fl_itemIds_do_not_clone = []

for itemId in wm_itemIds:
    item = source_gis.content.get(itemId)
    print("\nWorking on item:")
    print(item['id'], '"'+item['title']+'"', item['type'], item['owner'])
    print("type:", item['type'])
    print("typeKeywords:", item['typeKeywords'])
    
    # Is item a Web Map?
    if( item['type'] == 'Web Map' ):
        wm = mapping.WebMap(item)
        extract_fl_itemIds(wm.layers)
    # Is item a Web Scene?
    elif( item['type'] == 'Web Scene' ):
        ws = mapping.WebScene(item)
        extract_fl_itemIds(ws['operationalLayers'])
    # Something other than a Web Map or Web Scene, so ignore it.
    else:
        print("    >>>> IGNORE: Not a Web Map.")

In [ ]:
# Reduce lists to unique set of itemIds.
fl_itemIds_to_clone = list(set(fl_itemIds_to_clone))
fl_itemIds_do_not_clone = list(set(fl_itemIds_do_not_clone))

print("Feature Layers that will be cloned:")
for itemId in fl_itemIds_to_clone:
    print(itemId)
    
print("\nFeature Layers that will NOT be cloned:")
for itemId in fl_itemIds_do_not_clone:
    print(itemId)

## Clone Feature and Tile Layers

Start the cloning process by first cloning the appropriate feature and tile layers referenced by Web Maps and Web Scenes referenced in the StoryMap.

In [ ]:
# Clone Feature Layers and produce map of original to cloned itemIds.

fl_itemId_map = {}

print("\nOriginal itemId                  Cloned itemId")

for fl_itemId in fl_itemIds_to_clone:
    
    # Get original item
    original_item = source_gis.content.get(fl_itemId)
    
    # Is the item to clone a Hosted Tile Service?
    if( original_item.type == 'Map Service'):
        #print("Cloning Hosted Tilie Service:", fl_itemId)
        
        # Check for tile package as source
        related_items = original_item.related_items(
            rel_type = 'Service2Data', 
            direction = 'forward'
        )
        if(len(related_items) != 1 ):
            print(">>>> WARNING: Unexpected number of related items found!", len(related_items))
        else:
            # Is tile package owned by same user?
            related_item = related_items[0]
            if( related_item.owner != source_user ):
                print(">>>> WARNING: Source User does not own related item!", related_item.owner)
            else:
                # Clone tile package to target
                cloned_items = target_gis.content.clone_items(
                    items = [related_item],
                    search_existing_items = False
                )
                # Expecting only one cloned item when cloning the feature layer
                if( len(cloned_items) != 1 ):
                    print(">>>> WARNING: Unexpected number of clones produced!", len(cloned_items))
                    for item in cloned_items:
                        print(item['id'], '"'+item['title']+'"', item['type'], item['owner'])
                else:
                    # Publish tile package on target
                    cloned_item = cloned_items[0]
                    cloned_tile_service_item = cloned_item.publish(
                        output_type='Tiles',  
                        build_initial_cache=True
                    )

                    # Add itemId of published hosted tile layer on target to map
                    fl_itemId_map.update({fl_itemId:cloned_tile_service_item['id']})
                    cloned_tile_service_item
                    
                    print(fl_itemId, cloned_tile_service_item['id'], '"'+cloned_tile_service_item['title']+'"', cloned_tile_service_item['type'], cloned_tile_service_item['owner'])
        
                    # Add the Tag from above to the cloned items to help keep track of things.
                    cloned_tile_service_item['tags'].append(process_timestamp_tag)
                    result = cloned_tile_service_item.update(
                        item_properties = {
                            'tags': cloned_item['tags']
                        }
                    )
    # Otherwise, cloning a Hosted Feature Layer
    else:
        cloned_items = target_gis.content.clone_items(
            items = [source_gis.content.get(fl_itemId)],
            search_existing_items = False
        )

        # Expecting only one cloned item when cloning the feature layer
        if( len(cloned_items) != 1 ):
            print(">>>> WARNING: Unexpected number of clones produced!", len(cloned_items))
            for item in cloned_items:
                print(item['id'], '"'+item['title']+'"', item['type'], item['owner'])

        else:
            # Assume there is one cloned item and it is the feature layer
            cloned_item = cloned_items[0]
            print(fl_itemId, cloned_item['id'], '"'+cloned_item['title']+'"', cloned_item['type'], cloned_item['owner'])

            # Add the Tag from above to the cloned items to help keep track of things.
            cloned_item['tags'].append(process_timestamp_tag)
            result = cloned_item.update(
                item_properties = {
                    'tags': cloned_item['tags']
                }
            )
       
            # Add source/original and target/cloned feature layer's itemId to itemId map.
            fl_itemId_map.update({fl_itemId:cloned_item['id']})

In [ ]:
# Add mapping of non-cloned feature layers to point to themselves, so the references to the originals 
# will be preserved in the cloned StoryMap.
for fl_itemId in fl_itemIds_do_not_clone:
    fl_itemId_map.update({fl_itemId:fl_itemId})

# Show the whole feature layer itemId_map dictionary.
print("\nOriginal itemId                  Cloned itemId")
for k,v in fl_itemId_map.items():
    item = source_gis.content.get(k)
    print(k, v, '"'+item['title']+'"', item['type'], item['owner'])

## Clone Resources (e.g., Web Maps, Web Scenes)

The next step in the cloning process is to clone the appropriate resources referenced in the StoryMap.

In [ ]:
# Clone resources (i.e., Web Maps, Web Scenes) and generate a map of source itemIds to cloned itemIds.
# When cloning web maps, use the feature layer itemId map created above with the item_mapping parameter.
# At this point the list of resources to clone consists of only supported resource types (e.g, webmaps), as it was filtered above.
#
# There should only be one item returned by clone_items for each resource cloned..

wm_itemId_map = {}

print("\nOriginal itemId                  Cloned itemId")

for wm_itemId in wm_itemIds:
    cloned_items = target_gis.content.clone_items(
        items = [source_gis.content.get(wm_itemId)],
        item_mapping = fl_itemId_map,
        search_existing_items = False
    )
    
    # Expecting only one cloned item when cloning a Web Map or Web Scene.
    if( len(cloned_items) != 1 ):
        if( len(cloned_items) == 0 ):
            print(">>>> WARNING: No clones produced!")
        else:
            print(">>>> WARNING: Unexpected number (>1) of clones produced!", len(cloned_items))
            for item in cloned_items:
                print(item['id'], '"'+item['title']+'"', item['type'], item['owner'])
            
    # Assume there is one cloned item and it is the cloned web map.
    cloned_item = cloned_items[0]
    print()
    print(wm_itemId, cloned_item['id'], '"'+cloned_item['title']+'"', cloned_item['type'], cloned_item['owner'])
    
    # Add the Tag from above to the cloned items to help keep track of things.
    cloned_item['tags'].append(process_timestamp_tag)
    result = cloned_item.update(
        item_properties = {
            'tags': cloned_item['tags']
        }
    )
    
    # Add source/original and target/cloned webmaps' itemIds to itemId_map dictionary.
    wm_itemId_map.update({wm_itemId:cloned_item['id']})
    
    # If webmap is a Web Scene, then addtional steps are required to complete its clone.
    if( cloned_item['type'] == 'Web Scene' ):
        print("\nSwizzling layers of Web Scene:", wm_itemId, cloned_item['id'])
    
        # Get json data of original Web Scene
        data = source_gis.content.get(wm_itemId).get_data()
        
        # Swizzle feature Layer itemIDs as needed.
        for layer in data['operationalLayers']:
            if( layer['layerType'] == 'ArcGISFeatureLayer' and 'itemId' in layer.keys() ):
                print("\nArcGISFeatureLayer w/ itemId:", layer['title'])
                # Update both the layer's URL and itemId
                original_itemId = layer['itemId']
                original_url = layer['url']

                layer['url'] = target_gis.content.get(fl_itemId_map[layer['itemId']])['url'] + '/' + layer['url'].rpartition('/')[2]

                layer['itemId'] = fl_itemId_map[layer['itemId']]

                print("Mapped:", original_itemId, "to", layer['itemId'])
                print("Mapped:", original_url, "to", layer['url'])

            elif( layer['layerType'] == 'GroupLayer:', layer['title'] ):
                print("\nGroupLayer")
                for l in layer['layers']:
                    if( l['layerType'] == 'ArcGISFeatureLayer' and 'itemId' in l.keys() ):
                        print("\nGroup - ArcGISFeatureLayer w/ itemId:", l['title'])
                        # Update both the layer's URL and itemId
                        original_itemId = l['itemId']
                        original_url = l['url']

                        l['url'] = target_gis.content.get(fl_itemId_map[l['itemId']])['url'] + '/' + l['url'].rpartition('/')[2]

                        l['itemId'] = fl_itemId_map[l['itemId']]

                        print("Mapped:", original_itemId, "to", l['itemId'])
                        print("Mapped:", original_url, "to", l['url'])
        
        # Update json data of cloned Web Scene.
        result = cloned_item.update(data = data)
        print("Updated Web Scene data:", result)

## Clone StoryMap

The last thing that needs to be cloned is the StoryMap itself.

In [ ]:
# Clone the ArcGIS StoryMap from the source to the target
clones = target_gis.content.clone_items(
    items = [storymap_item],
    search_existing_items = False
)
# Cloned StoryMap should be the first and only item returned.
if( len(clones) != 1 ):
    if( len(clones) == 0 ):
        print(">>>> ERROR: No StoryMap clone produced!")
    else:
        print(">>>> ERROR: Unexpected number (>1) of clone items produced while cloning StoryMap!", len(clones))
        for item in clones:
            print(item['id'], '"'+item['title']+'"', item['type'], item['owner'])
            
storymap_clone = clones[0]
storymap_clone

In [ ]:
# Add the Tag from above to the cloned StoryMap to help keep track of things.
storymap_clone['tags'].append(process_timestamp_tag)
update_result = storymap_clone.update(
    item_properties = {
        #'title': '[Clone] ' + storymap_clone['title'],
        'tags': storymap_clone['tags']
    }
)
print("Update results:", update_result)

In [ ]:
# Update clone's url property to point at itself (clone_items initially leaves it pointing to the original StoryMap.)
# TODO: Check if this is still needed. Appears that clone_items() now correctly sets the url property on the clone for itself, rather than the original.
print("Original url:", storymap_clone.url)
result = storymap_clone.update(
    item_properties = {'url': 'https://storymaps.arcgis.com/stories/' + storymap_clone.id}
)
print("Update result:", result)
print("Updated url: ", storymap_clone.url)

## Update Cloned StoryMap's JSON with mapped itemIds

Swizzle the references in the Cloned StoryMap to point at the cloned Feature Layers, Web Maps, Web Scences, etc., rather than the originals.

In [ ]:
# Get draft JSON resource ID or file name.
cloned_smdraftresourceid = [x[18:] for x in storymap_clone['typeKeywords'] if x.startswith('smdraftresourceid:',0)][0]
cloned_smdraftresourceid

In [ ]:
# Function to replace itemIds in resource section of json using itemIds_map.
def replace_cloned_itemIds(data):
    for key, val in data['resources'].items():
        if( val['type'] == 'webmap' ):
            # Check to see if webmap's itemId is one that was cloned
            if( [k for k,v in wm_itemId_map.items() if k == val['data']['itemId']] ):
                # Replace itemId with cloned itemId.
                print("match", val['data']['itemId'], wm_itemId_map[val['data']['itemId']])
                val['data']['itemId'] = wm_itemId_map[val['data']['itemId']]
            else:
                print("no match", val['data']['itemId'])

In [ ]:
# Function to replace resource references (e.g., r-<itemId>) in json using itemIds_map.
def recursive_replace_reference(data, old, new):
    for k,v in data.items():
        if k == old:
            data[new] = data.pop(old)
        if v == old:
            data[k] = new
        elif isinstance(v,dict):
            recursive_replace_reference(v, old, new)

In [ ]:
# If StoryMap is published, then update published JSON.
if( any(status in ['smstatuspublished','smstatusunpublishedchanges'] for status in storymap_clone['typeKeywords']) ):
    
    # Get the current published json.
    clone_data_published = storymap_item.get_data()
    
    # Replace the source itemIds with the mapped target itemIds in the resource section of published json.
    print("Updating itemIds")
    replace_cloned_itemIds(clone_data_published)
    
    # Replace the itemIDs in the resource references.
    print("\nUpdating resource references")
    for k,v in wm_itemId_map.items():
        old = 'r-' + k
        new = 'r-' + v
        print("Replacing {0} with {1}".format(old, new))
        recursive_replace_reference(clone_data_published, old, new)
    
    # Update the cloned StoryMap's published json with the updated version.
    result = storymap_clone.update(
        data = clone_data_published
    )
    print("\nUpdating published data:", result)

In [ ]:
# If the draft version of the StoryMap was accessible, then also update the clone's draft json.
if( draft_accessible ):
    
    # Get the current draft json resource.
    wm_itemIds = get_resource_itemIds(storymap_item.resources.get(smdraftresourceid)['resources'])
    clone_data_draft = storymap_clone.resources.get(cloned_smdraftresourceid)

    # Replace the source itemIds with the mapped target itemIds in the resource section of draft json.
    print("Updating itemIds")
    replace_cloned_itemIds(clone_data_draft)

    # Replace the itemIDs in the resource references.
    print("\nUpdating resource references")
    for k,v in wm_itemId_map.items():
        old = 'r-' + k
        new = 'r-' + v
        print("Replacing {0} with {1}".format(old, new))
        recursive_replace_reference(clone_data_draft, old, new)

    # Remove current draft json resource.
    result = storymap_clone.resources.remove(cloned_smdraftresourceid)
    print("\nRemoving draft data:", result)
else:
    print("You do not have access to StoryMap's unpublished edits. Published JSON will be substituted for Draft JSON in cloned StoryMap.") 
    clone_data_draft = clone_data_published

# Add updated draft json resource back using same name.
result = storymap_clone.resources.add(
    file_name = cloned_smdraftresourceid,
    text = json.dumps(clone_data_draft)
)
print("\nAdding draft data:", result)

# Examine Cloned StoryMap

Done! View the final product.

In [ ]:
# Check out the Cloned Story to see if everything is okay.
storymap_clone

# Stop Here!
The content below is not part of the StoryMap cloning workflow, but can be helpful if troubleshooting is required.

In [ ]:
# Throw an exeception to prevent Run All Cells from going past this point. 
# (You don't want accidentally run the cells that would delete all the cloned content you just created!)
raise SystemExit("Stop!")

## Display StoryMap JSON
Print out the JSON data for the StoryMap, so that you can cut and paste it into a JSON viewer to make it easier to understand and navigate the hierarchy.

In [ ]:
print(json.dumps(storymap_item.resources.get(smdraftresourceid), indent=4))

In [ ]:
print(json.dumps(storymap_item.get_data(), indent=4))

In [ ]:
print(json.dumps(storymap_clone.resources.get(cloned_smdraftresourceid), indent=4))

In [ ]:
print(json.dumps(storymap_clone.get_data(), indent=4))

## Clean-up: Delete all the Cloned Content

In [ ]:
# Delete all the cloned content; identify it using the Tag assigned above.

# Get list of items with Tag.
print("Items to delete:")
items = target_gis.content.search('tags:' + process_timestamp_tag)
for item in items:
    print(item)
    
# Delete the list of items.
print("\nDeleting")
result = target_gis.content.delete_items(items)
print(result)

## Inspect an ArcGIS StoryMap's Properties

In [ ]:
storymap = source_gis.content.get('8607ff45996d4832894203d377ff3953')
storymap

In [ ]:
for k,v in storymap.items():
    print(k,v)